# Big Data project A.Y. 2024-2025

## Members

- Giovanni Antonioni
- Luca Rubboli - 0001083742

## Second job

In [3]:
import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.SparkSession
